# Analyzing Learning-Based Networked Systems with Formal Verification

## Building blocks implementation

This notebook contains the implementation of the building blocks described in section 4 of our paper. Those building blocks represent the encoding in MILP of simple, combinable properties.

**Note:** This is not an interactive notebook (do not try to run it).

### Constants and ranges

In [ ]:
# Encode a constant value
# Example: pensieve.py line 181

def encode_constant(variable, value):
    vars = [variable]
    coefs = [1]
    rhs = [value]
    tag_con = "fixed_feature_value_" + str(variable)

    ilpsolver.add_linear_constraint(rhs=rhs,
                                    senses='E',
                                    vars=vars,
                                    coefs=coefs,
                                    tag=tag_con)

In [ ]:
# Encode a range
# Example: pensieve.py line 195

def encode_range(variable, lb, ub):
    vars = [variable]
    coefs = [1]

    rhs = [lb]
    tag_con = "constraint_lower_bound_" + str(variable)
    ilpsolver.add_linear_constraint(rhs=rhs,
                                    senses='G',
                                    vars=vars,
                                    coefs=coefs,
                                    tag=tag_con)

    rhs = [ub]
    tag_con = "constraint_upper_bound_" + str(variable)
    ilpsolver.add_linear_constraint(rhs=rhs,
                                    senses='L',
                                    vars=vars,
                                    coefs=coefs,
                                    tag=tag_con)

### Linear relations

In [ ]:
# Encode a linear relation between variables
# Example: layers.py line 320

# variables * weights + bias = output

def encode_linear_relation(inputs, output, weights, bias):
    #sum a_i input_i + b = output
    #sum a_i input_i - output = -b
    vars = np.concatenate([inputs, [output]])
    coefs = np.concatenate([weights, [-1]])
    rhs = [float(-bias)]
    tag_con = "linear" + "_" + str(i) + "_" + str(output_idx)

    ilpsolver.add_linear_constraint(rhs=rhs,
                                    senses='E',
                                    vars=vars,
                                    coefs=coefs,
                                    tag=tag_con)

### Maximum value

In [ ]:
# Encode a variable as the maximum of a set
# Example: pensieve.py line 216

# variables[index] must be the maximum

def encode_maximum(variables, index):
    indicators, _ = ilpsolver.add_variables(label="q",
                                            ids=['prop_selected_output'],obj=0,
                                            lb=0, ub=1,
                                            type=self.ilpsolver.solver.variables.type.binary,
                                            sz=len(variables))
    
    for i in range(len(variables)):
        # inds[i] = 1 → vars[index] > vars[i]
        if i == index:
            continue
        vars = [variables[index], variables[i]]
        coefs = [1, -1]
        rhs = 0
        tag_con = "prop_maximum_ind_" + str(i)

        ilpsolver.add_indicator_constraint(indicator_vars=indicators[i],
                                           vars=vars,
                                           coefs=coefs,
                                           rhs=rhs,
                                           senses='G',
                                           complemented=0,
                                           tag=tag_con)

        # inds[i] = 0 → vars[index] < vars[i]
        coefs = [-1, 1]
        tag_con = "prop_maximum_inverted_ind_" + str(i)

        ilpsolver.add_indicator_constraint(indicator_vars=ind_vars,
                                           vars=vars,
                                           coefs=coefs,
                                           rhs=rhs,
                                           senses='G',
                                           complemented=1,
                                           tag=tag_con)
        
        # all indicators must be equal to 1 => sum must be equal to len(variables)-1
        coefs = [1] * len(variables)
        coefs[index] = 0
        vars = indicators
        rhs = [len(variables) - 1]
        tag_con = "prop_maximum_sum"

        ilpsolver.add_linear_constraint(rhs=rhs,
                                        senses='E',
                                        vars=vars,
                                        coefs=coefs,
                                        tag=tag_con)

### Discrete values

In [ ]:
# Encode a variable as having one out of a set of possible values
# Example: layers.py line 475

def encode_discrete_values(variable, values)
    indicators, _ = ilpsolver.add_variables(label="v",
                                            ids=[self.name],obj=0,
                                            lb=0,
                                            ub=1,
                                            type=ilpsolver.solver.variables.type.binary,
                                            sz=len(values))

    for i in range(len(values)):
        #ind = 1 → var = values[i]
        vars = [variable]
        coefs = [1]
        rhs = values[i]
        tag_con = "prop_discrete_ind_" + str(i)

        ilpsolver.add_indicator_constraint(indicator_vars=indicators[i],
                                           vars=vars,
                                           coefs=coefs,
                                           rhs=rhs,
                                           senses='E',
                                           complemented=0,
                                           tag=tag_con)

        # one indicator must be equal to 1 => sum must be equal to 1
        vars = indicators
        coefs = [1] * len(values)
        rhs = [1]
        tag_con = "prop_discrete_sum"
    
        ilpsolver.add_linear_constraint(rhs=rhs,
                                        senses='E',
                                        vars=vars,
                                        coefs=coefs,
                                        tag=tag_con)